In [1]:
!date

2025年  6月 10日 火曜日 09:20:21 JST



# カタカナから漢字・アルファベットを推測するモデルを学習する
オープンソース辞書及び，Wikipediaから作成した漢字とカタカナのペアデータを用いて，Transformerで，スクラッチで学習する。

入力の文字（カタカナ）は一文字ずつに分割，また，出力の文字（漢字・アルファベットなど）も一文字ずつに分割して，学習する。

データセットは，漢字からカタカナを推測するモデルのデータを，漢字とカナを逆にした者を作成して利用する。

## データ準備
データは，[dict](../dict)で作成した姓名辞書データのoss.json及びseimei.jsonと，Wikipediaから抜き出した，漢字姓名とカタカナ姓名（スペース区切りのもの）を用いて，学習に使用した。単語単位である姓名辞書データと，Wikipediaのスペースで区切られた姓名を用いることで，姓名で漢字とカナが入力された場合の学習を可能とした。

また，ひらがなとカタカナ，

In [2]:
# スクリプトで使う環境変数をセット
import os
VER="1.7o"
os.environ["VER"]=VER
os.environ['DIR']='dataset/dataset.'+os.environ['VER']
os.environ['MODEL']='model/model_r.'+os.environ['VER']
os.environ["LOG"]='logs/logs_r.'+os.environ['VER']
os.environ['RATIO']="0.01"

os.makedirs(os.environ['DIR'],exist_ok=True)
os.makedirs(os.environ['MODEL'],exist_ok=True)


In [3]:
# 訓練実行

!python ./char_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 32 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 145 \
  --device cuda  \
  --earlystop_patient 3 \
  --output_dir $MODEL \
  --tensorboard_logdir $LOG \
  --prefix translation \
  --source_lang kana \
  --target_lang kanji \
  --train_file $DIR/train.jsonl \
  --valid_file $DIR/valid.jsonl


num_epochs:145


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

## モデルを検証する
作成したモデルを，検証用データを用いて検証し，正解率を算出する


## パラメタ

- test_file    
jsonl形式の検証用データファイル

- model_file    
transformer_model.pyで作成したモデルファイル。

- outfile    
検証結果を格納するファイル

- device    
cpu限定

- nbest    
検証用データの漢字姓名を入力データとした時に，モデルが出力するカタカナ姓名を確率の高い方からnbest個出力する。beam_width以下。searchパラメタがbothかbeamの時有効

- beam_width    
ビームサーチのビーム幅。searchパラメタがbothかbeamの時有効

- max_len    
出力するカタカナ姓名の最大長さ

- search    
greedy,beam,bothから選択    
  - greedy    
貪欲法による探索    
  - beam    
ビームサーチによる探索    
  - both    
貪欲砲，ビームサーチ両方の探索    

In [ ]:
# 検証用データを実行


!python generate_batch.py \
  --test_file $DIR/test.jsonl \
  --model_file $MODEL/checkpoint_best.pt \
  --outfile $DIR/generate_r.txt \
  --device cpu \
  --nbest 5 \
  --beam_width 5 \
  --max_len 100 \
  --search greedy


In [ ]:
# 検証結果を評価
import pandas as pd
import os
dr=os.environ["DIR"]
df = pd.read_csv(f'{dr}/generate.txt',encoding='utf-8',sep="\t")
tgt = df['tgt'].tolist()
pred = df['pred'].tolist()
ok=0
for t,p in zip(tgt,pred):
    if t == p:
        ok+=1

print(f'acc={ok/len(tgt)}')


# モデルをJavaのDJL用に変換する
Pytorchで学習したモデルをJavaで使えるようにモデルの変換を行う。


In [ ]:
# カナから漢字
!python convert_jitscript.py  \
    --model_file=$MODEL/checkpoint_best.pt \
    --model_script=$MODEL/script_r_$VER.pt \
    --encoder=$MODEL/encoder_r_$VER.pt \
    --decoder=$MODEL/decoder_r_$VER.pt \
    --positional_encoding=$MODEL/positional_encoding_r_$VER.pt \
    --generator=$MODEL/generator_r_$VER.pt \
    --src_tok_emb=$MODEL/src_tok_emb_r_$VER.pt \
    --tgt_tok_emb=$MODEL/tgt_tok_emb_r_$VER.pt \
    --vocab_src=$MODEL/vocab_src_r_$VER.txt \
    --vocab_tgt=$MODEL/vocab_tgt_r_$VER.txt \
    --params=$MODEL/params_r_$VER.json \
    --source_lang=kana \
    --target_lang=kanji \
    --device=cpu


In [ ]:
!date